In [4]:
pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=edd690dad0f84d150165c007ff9d626ee2c8d84109fa9cec6527775589d14952
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [6]:
import chess
import random
from enum import Enum, auto

class GameState(Enum):
    INITIALIZING = auto()
    IN_PROGRESS = auto()
    CHECK = auto()
    CHECKMATE = auto()
    STALEMATE = auto()
    DRAW = auto()
    END = auto()

class GameEvent(Enum):
    START_GAME = auto()
    MOVE_MADE = auto()
    CHECK_OCCURS = auto()
    CHECKMATE_OCCURS = auto()
    STALEMATE_OCCURS = auto()
    DRAW_DECLARED = auto()
    END_GAME = auto()

class ChessGameFSM:
    def __init__(self):
        self.state = GameState.INITIALIZING
        self.board = chess.Board()
        print("Game initialized. Current state:", self.state)

    def on_event(self, event: GameEvent):
        print(f"Event: {event} | Current State: {self.state}")

        if self.state == GameState.INITIALIZING:
            if event == GameEvent.START_GAME:
                self.state = GameState.IN_PROGRESS
                print("Game started.")
                print(self.board)

        elif self.state == GameState.IN_PROGRESS:
            if event == GameEvent.CHECK_OCCURS:
                self.state = GameState.CHECK
            elif event == GameEvent.CHECKMATE_OCCURS:
                self.state = GameState.CHECKMATE
            elif event == GameEvent.STALEMATE_OCCURS:
                self.state = GameState.STALEMATE
            elif event == GameEvent.DRAW_DECLARED:
                self.state = GameState.DRAW

        elif self.state == GameState.CHECK:
            if event == GameEvent.MOVE_MADE:
                self.state = GameState.IN_PROGRESS
            elif event == GameEvent.CHECKMATE_OCCURS:
                self.state = GameState.CHECKMATE

        elif self.state in {GameState.CHECKMATE, GameState.STALEMATE, GameState.DRAW}:
            if event == GameEvent.END_GAME:
                self.state = GameState.END

        print("Transitioned to state:", self.state)

    def player_move_white(self, move_uci):
        """Player's move for White."""
        if self.state != GameState.IN_PROGRESS:
            print("Cannot make a move, the game is not in progress.")
            return

        move = chess.Move.from_uci(move_uci)
        if move not in self.board.legal_moves:
            print("Invalid move.")
            return

        self.board.push(move)
        print(f"White played: {move_uci}")
        print(self.board)

        if self.board.is_checkmate():
            self.on_event(GameEvent.CHECKMATE_OCCURS)
        elif self.board.is_stalemate():
            self.on_event(GameEvent.STALEMATE_OCCURS)
        elif self.board.is_check():
            self.on_event(GameEvent.CHECK_OCCURS)
        elif self.board.is_insufficient_material():
            self.on_event(GameEvent.DRAW_DECLARED)
        else:
            self.on_event(GameEvent.MOVE_MADE)
            self.bot_move_black()

    def bot_move_black(self):
        """Bot's move for Black."""
        if self.board.is_game_over():
            print("The game is over.")
            return

        legal_moves = list(self.board.legal_moves)
        bot_move = random.choice(legal_moves)
        self.board.push(bot_move)
        print(f"Black bot played: {bot_move}")
        print(self.board)

        if self.board.is_checkmate():
            self.on_event(GameEvent.CHECKMATE_OCCURS)
        elif self.board.is_stalemate():
            self.on_event(GameEvent.STALEMATE_OCCURS)
        elif self.board.is_check():
            self.on_event(GameEvent.CHECK_OCCURS)
        elif self.board.is_insufficient_material():
            self.on_event(GameEvent.DRAW_DECLARED)
        else:
            self.on_event(GameEvent.MOVE_MADE)

game = ChessGameFSM()
game.on_event(GameEvent.START_GAME)

Game initialized. Current state: GameState.INITIALIZING
Event: GameEvent.START_GAME | Current State: GameState.INITIALIZING
Game started.
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Transitioned to state: GameState.IN_PROGRESS


In [7]:
game.player_move_white("e2e4")

White played: e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS
Black bot played: g8f6
r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS


In [9]:
game.player_move_white("b1c3")

White played: b1c3
r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS
Black bot played: b8c6
r . b q k b . r
p p p p p p p p
. . n . . n . .
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS


In [10]:
game.player_move_white("g1f3")

White played: g1f3
r . b q k b . r
p p p p p p p p
. . n . . n . .
. . . . . . . .
. . . . P . . .
. . N . . N . .
P P P P . P P P
R . B Q K B . R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS
Black bot played: f6g4
r . b q k b . r
p p p p p p p p
. . n . . . . .
. . . . . . . .
. . . . P . n .
. . N . . N . .
P P P P . P P P
R . B Q K B . R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS


In [11]:
game.player_move_white("f3g5")

White played: f3g5
r . b q k b . r
p p p p p p p p
. . n . . . . .
. . . . . . N .
. . . . P . n .
. . N . . . . .
P P P P . P P P
R . B Q K B . R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS
Black bot played: g4e3
r . b q k b . r
p p p p p p p p
. . n . . . . .
. . . . . . N .
. . . . P . . .
. . N . n . . .
P P P P . P P P
R . B Q K B . R
Event: GameEvent.MOVE_MADE | Current State: GameState.IN_PROGRESS
Transitioned to state: GameState.IN_PROGRESS
